In [1]:
# essentially the jupyter notebook version of train.py; created to make model tuning faster.
# data loading is decoupled from training

import torch
import torch.nn as nn
from torch.optim.lr_scheduler import LambdaLR
from torch.utils.data import DataLoader

import matplotlib.pyplot as plt
import numpy as np

from preprocessing.main import main, get_edf_files
from crnn_tf_v3 import CRNN
from dataloader import ANNEDataset

import json
import math
import time
import random

random.seed(42)
torch.manual_seed(42)
timestr = time.strftime("%Y%m%d-%H%M%S")

N_CLASSES = 3


class CosineWithWarmupLR(LambdaLR):
    def __init__(self, optimizer, warmup_epochs, max_epochs, max_lr=0.001, min_lr=0.0001):
        self.warmup_epochs = warmup_epochs
        self.max_epochs = max_epochs
        # self.base_lr = base_lr
        warmup_scheduler = lambda epoch: epoch / warmup_epochs if epoch < warmup_epochs else \
            (0.5 * (1.0 + math.cos(math.pi * (epoch - warmup_epochs) / (max_epochs - warmup_epochs))) if epoch < max_epochs else min_lr/max_lr)

        super(CosineWithWarmupLR, self).__init__(optimizer, lr_lambda=warmup_scheduler)


def save_strings_to_json(strings_list, filename):
    data = {"strings": strings_list}

    with open(filename, "w") as json_file:
        json.dump(data, json_file, indent=4)

def train_model(model, optimizer, train_loaders, test_loaders, lr_scheduler, epochs=100, print_every=10):

    # Using GPUs in PyTorch is pretty straightforward
    if torch.cuda.is_available():
        print("Using cuda")
        use_cuda = True
        device = torch.device("cuda")
    else:
        device = "cpu"

    if N_CLASSES == 3:
        xentropy_weight = torch.tensor([1 / 27 ** 1.5, 1 / 62 ** 1.5, 1 / 11 ** 1.5]).to(device)
    else:
        xentropy_weight = torch.tensor([1 / 27 ** 1.75 , 1 / 73 ** 1.75]).to(device)

    criterion = nn.CrossEntropyLoss(weight=xentropy_weight)
    train_accs = []
    test_accs = []
    train_losses = []
    test_losses = []
    learning_rates = []
    # Early return parameters
    patience = 1000       # we early return if there is no improvement after patience number of epochs
    counter = 0
    # min_delta = 0.01    # at least 1% accuracy increase is needed to count it as an improvement
    min_delta = 0
    best_test_loss = float("inf")

    # Move the model to GPU, if available
    model.to(device)
    model.train()

    for epoch in range(epochs):
        xentropy_loss_total = 0.
        correct = 0.
        total = 0.
        total_trainer_len = 0
        for train_loader in train_loaders:
            for i, (inputs, inputs_freq, inputs_scl, labels, lengths) in enumerate(train_loader):
                model.zero_grad()
                inputs = inputs.to(device)
                inputs_freq = inputs_freq.to(device)
                inputs_scl = inputs_scl.to(device)
                labels = labels.to(device)
                # inputs = inputs.view(inputs.size(0), -1)  # Flatten input from [batch_size, 1, 28, 28] to [batch_size, 784]
                pred = model(inputs, inputs_freq, inputs_scl)
                xentropy_loss = criterion(pred, labels)
                xentropy_loss.backward()

                xentropy_loss_total += xentropy_loss.item()

                # Calculate running average of accuracy
                pred = torch.max(pred.data, 1)[1]
                total += labels.size(0)
                correct += (pred == labels.data).sum().item()

            optimizer.step()
            total_trainer_len += len(train_loader)

        # lr_scheduler.step(epoch + i / len(train_loader))    # Important: use this for CosineAnnealingWarmRestarts
        lr_scheduler.step()  # use this for CyclicLR
        current_lr = lr_scheduler.get_last_lr()[0]
        learning_rates.append(current_lr)
        print(f"Learning Rate {current_lr}")

        accuracy = correct / total
        avg_xentropy_loss = xentropy_loss_total / total_trainer_len

        test_acc_sum = 0
        test_loss_sum = 0

        for test_loader in test_loaders:
            test_acc, test_loss = evaluate(model, test_loader, criterion, device)
            test_acc_sum += test_acc
            test_loss_sum += test_loss

        test_acc = test_acc_sum / len(test_loaders)
        test_loss = test_loss_sum / len(test_loaders)

        if epoch % print_every == 0:
            print("Epoch {}, Train acc: {:.2f}%, Test acc: {:.2f}%".format(epoch, accuracy * 100, test_acc * 100))
            print("Epoch {}, Train loss: {:.2f}, Test loss: {:.2f}".format(epoch, avg_xentropy_loss, test_loss))

        train_accs.append(accuracy)
        test_accs.append(test_acc)
        train_losses.append(avg_xentropy_loss)
        test_losses.append(test_loss)

        # Check for early stopping
        if test_loss < best_test_loss - min_delta:
            best_test_loss = test_loss
            model_scripted = torch.jit.script(model)
            model_scripted.save(f"checkpoints/es_{timestr}.pt")
            print("saved new model")
            counter = 0
        else:
            counter += 1
            if counter >= patience:
                print("Early stopping triggered.")
                break

    return train_accs, test_accs, train_losses, test_losses, learning_rates


def evaluate(model, loader, criterion, device):
    model.eval()  # Change model to 'eval' mode (BN uses moving mean/var).
    correct = 0.
    total = 0.
    val_loss = 0.
    for i, (inputs, inputs_freq, inputs_scl, labels, lengths) in enumerate(loader):
        with torch.no_grad():
            inputs = inputs.to(device)
            inputs_freq = inputs_freq.to(device)
            labels = labels.to(device)
            # inputs = inputs.view(inputs.size(0), -1)
            pred = model(inputs, inputs_freq, inputs_scl)
            xentropy_loss = criterion(pred, labels)
            val_loss += xentropy_loss.item()

        pred = torch.max(pred.data, 1)[1]
        total += labels.size(0)
        correct += (pred == labels).sum().item()

    val_acc = correct / total
    val_loss = val_loss / len(loader)
    model.train()
    return val_acc, val_loss


def plot_grad_histograms(grad_list, epoch, init=False):
    fig, ax = plt.subplots(nrows=1, ncols=len(grad_list), figsize=(5 * len(grad_list), 5))
    for i, grad in enumerate(grad_list):
        plt.subplot(1, len(grad_list), i + 1)
        plt.hist(grad)
        if init:
            plt.title("Grads for Weights (Layer {}-{}) Init".format(i, i + 1))
        else:
            plt.title("Grads for Weights (Layer {}-{}) Epoch {}".format(i, i + 1, epoch))
    plt.show()


def plot_act_histograms(act_list, epoch, init=False):
    fig, ax = plt.subplots(nrows=1, ncols=len(act_list), figsize=(5 * len(act_list), 5))
    for i, act in enumerate(act_list):
        plt.subplot(1, len(act_list), i + 1)
        plt.hist(act)
        if init:
            plt.title("Activations for Layer {} Init".format(i + 1))
        else:
            plt.title("Activations for Layer {} Epoch {}".format(i + 1, epoch))
    plt.show()

In [2]:
# Check gpu availability
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

torch.cuda.empty_cache()
# Load data:
train_list = get_edf_files("/mnt/Common/data")

validation_list = random.sample(train_list, 20)
print(validation_list)
save_strings_to_json(validation_list, "./validation.json")
#
# train_list = train_list_[:2]
# print(train_list)
# validation_list = [train_list_[1]]

train_dataloaders = []
valid_dataloaders = []
for path in train_list:
    # try:
        X, X_freq, X_scl, t = main(path)
        # for binary classification
        if N_CLASSES == 2:
            t = np.where(t == 2, 1, t)
        # print(t)
        dataset = ANNEDataset(X, X_freq, np.zeros(shape = (len(X), 1)), t, device)
        size = len(X)
        if path not in validation_list:
            train_dataloaders.append(DataLoader(dataset=dataset, batch_size=size))

        else:
            valid_dataloaders.append(DataLoader(dataset=dataset, batch_size=size))
    # except:
    #     print(f"Something went wrong for file {path}")
# random.shuffle(train_list)

['/mnt/Common/data/20-10-02-21_48_39.C1390.L1217.205-annotated.edf', '/mnt/Common/data/20-02-10-20_07_16.C823.L931.7-annotated.edf', '/mnt/Common/data/21-03-22-22_36_53.C1442.L1215.261-annotated.edf', '/mnt/Common/data/21-02-18-21_07_23.C1442.L1215.250-annotated.edf', '/mnt/Common/data/21-01-12-21_30_33.C1442.L1215.238-annotated.edf', '/mnt/Common/data/20-10-23-21_08_59.C1442.L1215.213-annotated.edf', '/mnt/Common/data/20-09-10-23_48_07.C1425.L1205.194-annotated.edf', '/mnt/Common/data/20-08-24-21_44_04.C1419.L1362.186-annotated.edf', '/mnt/Common/data/21-09-15-22_59_43.C1390.L3562.316-annotated.edf', '/mnt/Common/data/20-02-19-19_54_00.C823.L931.10-annotated.edf', '/mnt/Common/data/20-02-18-20_05_51.C823.L931.9-annotated.edf', '/mnt/Common/data/20-08-26-22_34_39.C1390.L1217.188-annotated.edf', '/mnt/Common/data/21-01-06-22_45_53.C1425.L1205.236-annotated.edf', '/mnt/Common/data/21-02-01-21_36_56.C1425.L1205.244-annotated.edf', '/mnt/Common/data/23-05-03-21_29_35.C4359.L3786.589-annota

/mnt/Common/Projects/ANNE_dl/preprocessing/main.py:71: RuntimeWarning: Physical range is not defined in following channels:
PLM.events
  data = mne.io.read_raw_edf(path)


Extracting EDF parameters from /mnt/Common/data/20-01-14-20_23_27.C823.L775.4-annotated.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /mnt/Common/data/20-01-15-20_13_00.C823.L775.5-annotated.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


/mnt/Common/Projects/ANNE_dl/preprocessing/main.py:71: RuntimeWarning: Physical range is not defined in following channels:
PLM.events
  data = mne.io.read_raw_edf(path)


Extracting EDF parameters from /mnt/Common/data/20-02-04-20_34_29.C823.L931.6-annotated.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /mnt/Common/data/20-02-10-20_07_16.C823.L931.7-annotated.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


...

/mnt/Common/Projects/ANNE_dl/preprocessing/main.py:71: RuntimeWarning: Physical range is not defined in following channels:
SpO2.events
  data = mne.io.read_raw_edf(path)


Extracting EDF parameters from /mnt/Common/data/23-04-20-21_40_19.C4359.L3786.580-annotated.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


/home/a663e-36z/anaconda3/lib/python3.9/site-packages/mne/io/edf/edf.py:673: RuntimeWarning: invalid value encountered in subtract
  edf_info["physical_min"] - edf_info["digital_min"] * edf_info["cal"]
/home/a663e-36z/anaconda3/lib/python3.9/site-packages/mne/io/edf/edf.py:407: RuntimeWarning: invalid value encountered in multiply
  ch_data = ch_data * cal[orig_idx]


Extracting EDF parameters from /mnt/Common/data/23-04-24-20_54_29.C4181.L3766.582-annotated.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /mnt/Common/data/23-04-26-20_42_32.C4181.L3766.584-annotated.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /mnt/Common/data/23-05-03-21_29_35.C4359.L3786.589-annotated.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


/mnt/Common/Projects/ANNE_dl/preprocessing/main.py:71: RuntimeWarning: Physical range is not defined in following channels:
PLM.events
  data = mne.io.read_raw_edf(path)


In [3]:
model = CRNN(num_classes=N_CLASSES, in_channels=X.shape[1], in_channels_f=X_freq.shape[1], in_channels_s=0, model='gru')
#
# MODEL_PATH = ""
# model = torch.load(MODEL_PATH)
# Initialize dataloaders
# train_dataset = ANNEDataset(X1, X1f, X1s, t1, device)
# train_dataloader = DataLoader(dataset=train_dataset, batch_size=4096)
# val_dataset = ANNEDataset(X2, X2f, X2s, t2, device)
# val_dataloader = DataLoader(dataset=val_dataset)

# Visualize model
# dummy_input = torch.randn(1024, 6, 25 * 30)
# torch.onnx.export(model, dummy_input, "./model.onnx")

# Train model:
learning_rate = 0.0000675
epochs = 400
# dummy_input = torch.randn(4096, X.shape[1], 25*30)
# dummy_input_freq = torch.randn(4096, X_freq.shape[1], X_freq.shape[2])
# dummy_input_scl = torch.randn(4096, X_scl.shape[1], X_scl.shape[2])
# torch.onnx.export(model, dummy_input, "./model.onnx")

# Train model:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, betas=(0.9, 0.95), weight_decay=0.1)
# Create the learning rate scheduler
scheduler = CosineWithWarmupLR(optimizer, warmup_epochs=10, max_epochs=300, max_lr=learning_rate, min_lr=0.000001)
# scheduler = CyclicLR(optimizer, max_lr = 0.01, base_lr =0.0000001, step_size_up=15, step_size_down=20,
# gamma=0.85, cycle_momentum=False, mode="triangular2") Run the training loop
train_accs, test_accs, train_losses, test_losses, learning_rates = train_model(model, optimizer, train_dataloaders,
                                                                               valid_dataloaders,
                                                                               scheduler,
                                                                               epochs=epochs,
                                                                               print_every=1)
model_scripted = torch.jit.script(model)
model_scripted.save(f"checkpoints/model_{timestr}.pt")
print("Model Saved")

plt.plot(train_losses)
plt.plot(test_losses)
plt.title("loss")
plt.show()

plt.plot(train_accs)
plt.plot(test_accs)
plt.title("accuracy")
plt.show()

plt.plot(learning_rates)
plt.plot(learning_rates)
plt.title("learning_rates")
plt.show()

Using cuda
Learning Rate 6.750000000000001e-06
Epoch 0, Train acc: 30.42%, Test acc: 29.11%
Epoch 0, Train loss: 1.25, Test loss: 1.19
saved new model
Learning Rate 1.3500000000000001e-05
Epoch 1, Train acc: 31.70%, Test acc: 34.66%
Epoch 1, Train loss: 1.21, Test loss: 1.15
saved new model
Learning Rate 2.025e-05
Epoch 2, Train acc: 31.33%, Test acc: 38.68%
Epoch 2, Train loss: 1.17, Test loss: 1.12
saved new model
Learning Rate 2.7000000000000002e-05
Epoch 3, Train acc: 29.29%, Test acc: 39.52%
Epoch 3, Train loss: 1.15, Test loss: 1.11
saved new model
Learning Rate 3.375e-05
Epoch 4, Train acc: 29.25%, Test acc: 40.76%
Epoch 4, Train loss: 1.13, Test loss: 1.10
saved new model
Learning Rate 4.05e-05
Epoch 5, Train acc: 29.36%, Test acc: 41.51%
Epoch 5, Train loss: 1.12, Test loss: 1.09
saved new model
Learning Rate 4.7249999999999997e-05
Epoch 6, Train acc: 29.43%, Test acc: 42.15%
Epoch 6, Train loss: 1.11, Test loss: 1.09
saved new model
Learning Rate 5.4000000000000005e-05
Epoch 